In [1]:
import csv,sys
import pandas as pd
import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_curve
data=[]

In [6]:
def Calc_equal_err_rate(realUser_scores,fakeUser_scores):
    labels = [0]*len(realUser_scores) + [1]*len(fakeUser_scores)
    fpr, tpr, thresholds = roc_curve(labels, realUser_scores + fakeUser_scores)
    # false alarm rate = fpr (false positive rate) 
    false_alarm_rates = fpr
    
    # hitrate = tpr (true positive rate)
    miss_rate= 1 - tpr
    all_dist = miss_rate - false_alarm_rates
    
    idx1 = np.argmin(all_dist[all_dist >=0 ])
    idx2 = np.argmax(all_dist[all_dist < 0])
    
    x_ptr = [miss_rate[idx1], false_alarm_rates[idx1]]
    y_ptr = [miss_rate[idx2], false_alarm_rates[idx2]]
    m = ( x_ptr[0] -x_ptr[1] ) / ( y_ptr[1] - x_ptr[1] - y_ptr[0] + x_ptr[0] )
   
    equal_err_rate = x_ptr[0] + m * ( y_ptr[0] - x_ptr[0] )
#     print (round(equal_err_rate,4))
    
    return equal_err_rate
    
    

In [7]:
def SVM(person):
    realUser_scores = []                         # real user score
    fakeUser_scores = []                         # imposter user score
    
    realUser_data = data.loc[data.subject == person, "H.period":"H.Return"]
    fakeUser_data = data.loc[data.subject != person, :]
    
    real_train = np.array((realUser_data[:200]).values)
    
    real_test = np.array((realUser_data[200:]).values)
    
    fake_test = np.array((fakeUser_data.groupby("subject").head(5).loc[:, "H.period":"H.Return"]).values)

    clf = OneClassSVM(kernel='rbf',gamma=26)
    clf.fit(real_train)
    realUser_scores =list( -clf.decision_function(real_test))
    fakeUser_scores = list(-clf.decision_function(fake_test))
    
    return Calc_equal_err_rate(realUser_scores,fakeUser_scores)


In [8]:
#     print (realUser_scores, fakeUser_scores)
#     print(np.shape(realUser_data))
#     print(np.shape(fakeUser_data))
#     print(np.shape(real_train))
#     print(np.shape(real_test))
#     print(np.shape(fake_test))

def main():
    global data
    data = pd.read_csv("./dataset/DSL-StrongPasswordData.csv")
    subjects = data["subject"].unique()
    header = [c for c in data]
    equal_err_rate =[]
    for person in subjects:
        equal_err_rate.append(SVM(person))
    
    print (np.mean(equal_err_rate), np.std(equal_err_rate))

In [9]:
if __name__ == main():
    main()

0.120650799483 0.0586694992117
